In [57]:
from datasets import Dataset
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
import torch
import pickle

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [59]:
def preprocess(examples, focused_trait="conscientiousness"):
    label = examples[focused_trait]
    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    examples['label'] = float(label)
    return examples



def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)    
    
    return {"mse": mse, "mae": mae, "r2": r2}



class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss


In [6]:
traits = ['conscientiousness', 'openness', 'neuroticism','agreeableness', 'extraversion']

# using disilroberta as a quick baseline model
# https://huggingface.co/docs/transformers/tasks/sequence_classification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=1
)


for focused_trait in traits:
    print(f"start working on f{focused_trait}")

    
    training_feature_df = pd.read_parquet("liwc_training_dataset_350words.parquet")
    ds = Dataset.from_dict(training_feature_df.to_dict('list'))
    tokenized_ds = ds.map(preprocess, fn_kwargs={"focused_trait": focused_trait})#remove_columns=['post_count','word_count','user'])
    train_test = tokenized_ds.train_test_split(test_size=0.3, seed=42)
    test_eval = train_test['test'].train_test_split(test_size=0.5, seed=42)


    LEARNING_RATE = 2e-5
    MAX_LENGTH = 256
    BATCH_SIZE = 16
    EPOCHS = 20

    training_args = TrainingArguments(
        output_dir=f"../models/fine-tuned-regression-{focused_trait}-training-text-350",
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=EPOCHS,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        metric_for_best_model="r2",
        load_best_model_at_end=True,
        weight_decay=0.01,
    )


    trainer = RegressionTrainer(
        model=model,
        args=training_args,
        train_dataset=train_test["train"],
        eval_dataset=test_eval["train"],
        compute_metrics=compute_metrics_for_regression,
    )

    trainer.train()

    trainer.eval_dataset=test_eval["test"]
    eval_ret = trainer.evaluate()

    # save dictionary to person_data.pkl file
    with open(f'../models/fine-tuned-regression-{focused_trait}-training-text-350/eval_result.pkl', 'wb') as fp:
        pickle.dump(eval_ret, fp)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


start working on fconscientiousness


Map: 100%|██████████| 1804/1804 [00:02<00:00, 756.61 examples/s]
/Users/bohaocao/anaconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  5%|▌         | 79/1580 [01:26<25:33,  1.02s/it]

{'eval_loss': 2172.096435546875, 'eval_mse': 2172.09619140625, 'eval_mae': 45.63023376464844, 'eval_r2': -23.080770548738933, 'eval_runtime': 5.1912, 'eval_samples_per_second': 52.204, 'eval_steps_per_second': 3.275, 'epoch': 1.0}


 10%|█         | 158/1580 [02:52<22:23,  1.06it/s]

...

100%|██████████| 1580/1580 [30:08<00:00,  1.11s/it]

{'eval_loss': 31.108734130859375, 'eval_mse': 31.10873794555664, 'eval_mae': 4.342731952667236, 'eval_r2': 0.6952215010735697, 'eval_runtime': 5.3228, 'eval_samples_per_second': 50.913, 'eval_steps_per_second': 3.194, 'epoch': 20.0}


100%|██████████| 1580/1580 [30:09<00:00,  1.15s/it]


{'train_runtime': 1809.8704, 'train_samples_per_second': 13.946, 'train_steps_per_second': 0.873, 'train_loss': 10.058879919897151, 'epoch': 20.0}


100%|██████████| 17/17 [00:04<00:00,  3.49it/s]


In [8]:
import pickle
import pandas as pd
keys = eval_ret.keys()
d = {}
for k in keys:
    d[k]=[]
d['trait'] = []

traits = ['conscientiousness', 'openness', 'neuroticism','agreeableness', 'extraversion']
for focused_trait in traits:
        with open(f'../models/fine-tuned-regression-{focused_trait}-training-text-350/eval_result.pkl', 'rb') as fp:
                eval_ret = pickle.load(fp)
                for k in eval_ret.keys():
                        d[k].append(eval_ret[k])
                d['trait'].append(focused_trait)
        

In [10]:
pd.DataFrame(d).to_csv("eval_result_text_350.csv")

In [11]:
# compare previous vs new model
new_training = pd.DataFrame(d)
old_training = pd.read_csv("eval_result.csv")

In [19]:
col = ['trait','eval_r2']
compare_df = old_training[col].merge(new_training[col], on="trait",suffixes=("_120","_350"))
compare_df['abs_chg'] = compare_df['eval_r2_350']-compare_df['eval_r2_120']
compare_df['pct_chg'] = (compare_df['eval_r2_350']-compare_df['eval_r2_120'])/compare_df['eval_r2_120']*100

In [21]:
compare_df.to_csv("improvement.csv")

## batch inference

In [45]:
from collections import defaultdict
from transformers import AutoTokenizer
from datasets import Dataset
import pandas as pd

# using disilroberta as a quick baseline model
# https://huggingface.co/docs/transformers/tasks/sequence_classification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)


def preprocess_for_inference_dataset(examples):
    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    return examples


all_user_text_df = pd.read_parquet("liwc_dataset_user_max_425.parquet")
ds_inference = Dataset.from_dict(all_user_text_df.to_dict('list'))
tokenized_ds_inference = ds_inference.map(preprocess_for_inference_dataset, remove_columns=['post_count','word_count','user'])

FileNotFoundError: [Errno 2] No such file or directory: 'liwc_dataset_user_max_425.parquet'

In [36]:
# loading the model previously trained
model_paths = [
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-agreeableness-training-text-350/checkpoint-1343",
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-conscientiousness-training-text-350/checkpoint-1580",
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-extraversion-training-text-350/checkpoint-1343",
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-neuroticism-training-text-350/checkpoint-711",
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-openness-training-text-350/checkpoint-1343"]
d = {}

for model_path in model_paths:
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    trait = model_path.split(sep="/")[-2].split(sep="-")[-4]
    # arguments for Trainer
    test_args = TrainingArguments(
        output_dir = model_path,
        do_train = False,
        do_predict = True,
        per_device_eval_batch_size = 50,   
        dataloader_drop_last = False    
    )

    # init trainer
    trainer = Trainer(
        model = model, 
        args = test_args, 
        compute_metrics = compute_metrics_for_regression
    )

    predictions = trainer.predict(tokenized_ds_inference)

    predictions_df = pd.DataFrame(predictions.predictions)
    d[trait]=predictions_df[0].tolist()

100%|██████████| 286/286 [04:41<00:00,  1.02it/s]


In [37]:
predictions_df = pd.DataFrame(d)

In [38]:
predictions_df

,agreeableness,conscientiousness,extraversion,neuroticism,openness
0,62.584175,53.016537,62.961075,33.943050,48.623798
1,52.349983,54.205620,53.372646,53.894497,53.370544
2,64.413727,60.303745,63.131458,32.397472,55.495106
3,55.095215,45.208984,56.746407,36.411385,46.324245
4,38.849571,48.048588,44.902611,51.545254,50.926304
...,...,...,...,...,...
14282,71.958252,60.066216,69.318344,31.210674,45.827385
14283,62.504547,57.361919,62.434406,38.509449,44.573627
14284,59.323158,59.692585,68.001259,41.594982,45.992832
14285,55.709034,44.886978,54.742641,43.495834,44.475941


In [39]:
all_user_text_df

,user,text,post_count,word_count
0,ariannyceleste,Raidens Mom 👶🏽🐶🐶🧿 Founder @girlfriendbox @them...,25,264
1,sarahstage,Boy Mom 👶🏻👦🏻 Wife 💃🏻 & Online Fitness Coach 💪🏼...,25,412
2,beauty_nurse_elizabeth,Aesthetic RN-BSN •Injection Artist 🌹 •Natural ...,25,383
3,carlyrbel,One of a kind ✨ • @carlybelx • shop @carlyclub...,25,142
4,biolayne,PhD Nutrition Science 💍@hollytbaxter @carbondi...,25,249
...,...,...,...,...
14282,lianev,Business inquiries Jennifer@thejgoagency.com T...,25,396
14283,kuz,Vino @lakers inquiries@kylekuzma.com @puma Ath...,25,281
14284,tanamongeau,BLACK. LIVES. MATTER. LINK IN BIO TO HELP; @th...,25,379
14285,lilyachty,no stylist; Itâ€™s us!; no stylist; It’s us!;;...,10,72


In [40]:
user_text_prediction_df = all_user_text_df.merge(predictions_df,left_index=True, right_index=True)

In [41]:
user_text_prediction_df.to_csv("all_predictions_word_350.csv")

In [43]:
# Convert the 0-100 numeric score into 1-10 categories.
traits = ['conscientiousness', 'openness', 'neuroticism','agreeableness', 'extraversion']
for trait in traits:
    user_text_prediction_df[f'{trait}_prediction'] = pd.cut(user_text_prediction_df[trait], 10, labels=[n for n in range(1, 11)])
#pd.cut(np.array([1, 7, 5, 4, 6, 3]), 3)

In [46]:
columns = ['user','conscientiousness_prediction', 'openness_prediction', 'neuroticism_prediction','agreeableness_prediction', 'extraversion_prediction']
user_text_prediction_df.rename(columns={
    "conscientiousness_categorized":"conscientiousness_prediction",
    "openness_categorized":"openness_prediction",
    "neuroticism_categorized":"neuroticism_prediction",
    "agreeableness_categorized":"agreeableness_prediction",
    "extraversion_categorized":"extraversion_prediction"},inplace=True)


In [67]:
import os 
import pandas as pd

files = os.listdir("../source_data")
files.sort()

df_women_event = pd.read_csv(os.path.join("../source_data", files[0]), sep="\t")
df_buy_insta_accounts_add_on = pd.read_csv(os.path.join("../source_data", files[1]), sep="\t")
df_buy_biz = pd.read_csv(os.path.join("../source_data", files[2]), sep="\t")
df_buy_insta_accounts = pd.read_csv(os.path.join("../source_data", files[3]), sep="\t")
df_women_event_mined = pd.read_csv(os.path.join("../source_data", files[4]), sep="\t")

In [52]:
user_text_prediction_df

,user,text,post_count,word_count,agreeableness,conscientiousness,extraversion,neuroticism,openness,conscientiousness_prediction,openness_prediction,neuroticism_prediction,agreeableness_prediction,extraversion_prediction
0,ariannyceleste,Raidens Mom 👶🏽🐶🐶🧿 Founder @girlfriendbox @them...,25,264,62.584175,53.016537,62.961075,33.943050,48.623798,9,8,5,9,8
1,sarahstage,Boy Mom 👶🏻👦🏻 Wife 💃🏻 & Online Fitness Coach 💪🏼...,25,412,52.349983,54.205620,53.372646,53.894497,53.370544,9,9,9,7,6
2,beauty_nurse_elizabeth,Aesthetic RN-BSN •Injection Artist 🌹 •Natural ...,25,383,64.413727,60.303745,63.131458,32.397472,55.495106,10,9,5,9,8
3,carlyrbel,One of a kind ✨ • @carlybelx • shop @carlyclub...,25,142,55.095215,45.208984,56.746407,36.411385,46.324245,8,7,6,7,7
4,biolayne,PhD Nutrition Science 💍@hollytbaxter @carbondi...,25,249,38.849571,48.048588,44.902611,51.545254,50.926304,8,8,9,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14282,lianev,Business inquiries Jennifer@thejgoagency.com T...,25,396,71.958252,60.066216,69.318344,31.210674,45.827385,10,7,4,10,9
14283,kuz,Vino @lakers inquiries@kylekuzma.com @puma Ath...,25,281,62.504547,57.361919,62.434406,38.509449,44.573627,10,7,6,9,8
14284,tanamongeau,BLACK. LIVES. MATTER. LINK IN BIO TO HELP; @th...,25,379,59.323158,59.692585,68.001259,41.594982,45.992832,10,7,7,8,9
14285,lilyachty,no stylist; Itâ€™s us!; no stylist; It’s us!;;...,10,72,55.709034,44.886978,54.742641,43.495834,44.475941,7,7,7,7,7


In [53]:
df_women_event = df_women_event.merge(df_women_event[columns], left_on="username", right_on="user",how="left")

In [54]:
df_buy_insta_accounts_add_on=df_buy_insta_accounts_add_on.merge(user_text_prediction_df[columns], left_on="username", right_on="user",how="left")

In [55]:
df_buy_biz=df_buy_biz.merge(user_text_prediction_df[columns], left_on="username", right_on="user",how="left")

In [56]:
df_buy_insta_accounts = df_buy_insta_accounts.merge(user_text_prediction_df[columns], left_on="username", right_on="user",how="left")

In [57]:
df_women_event_mined = df_women_event_mined.merge(user_text_prediction_df[columns], left_on="username", right_on="user",how="left")

In [58]:
df_women_event.to_csv("df_women_event_prediction_350.csv",index=False)
df_buy_insta_accounts_add_on.to_csv("df_buy_insta_accounts_add_on_prediction_350.csv",index=False)
df_buy_biz.to_csv("df_buy_biz_prediction_350.csv",index=False)
df_buy_insta_accounts.to_csv("df_buy_insta_accounts_prediction_350.csv",index=False)
df_women_event_mined.to_csv("df_women_event_mined_prediction_350.csv",index=False)

## Do inference for users without posts/captions

In [9]:
import pandas as pd
post_df = pd.read_parquet("../concat_feature.parquet")

In [14]:
import os

files = os.listdir("../source_data")
files.sort()

df_women_event = pd.read_csv(os.path.join("../source_data", files[0]), sep="\t")
df_buy_insta_accounts_add_on = pd.read_csv(os.path.join("../source_data", files[1]), sep="\t")
df_buy_biz = pd.read_csv(os.path.join("../source_data", files[2]), sep="\t")
df_buy_insta_accounts = pd.read_csv(os.path.join("../source_data", files[3]), sep="\t")
df_women_event_mined = pd.read_csv(os.path.join("../source_data", files[4]), sep="\t")

In [35]:
df_women_event.columns

Index(['username', 'media_count', 'full_name', 'following_count',
       'follower_count', 'first_name', 'external_url', 'engagement_percent',
       'email', 'contact_phone_number', 'cityName', 'category', 'biography',
       'address_street', 'MostRecentPostDate', 'AvgLikes', 'AvgComments'],
      dtype='object')

In [42]:
missing_users_df = pd.concat([df_women_event[~df_women_event.username.isin(user_text_prediction_df.user)][['username', 'biography']],
           df_buy_insta_accounts_add_on[~df_buy_insta_accounts_add_on.username.isin(user_text_prediction_df.user)][['username', 'biography']],
           df_buy_biz[~df_buy_biz.username.isin(user_text_prediction_df.user)][['username', 'biography']],
           df_buy_insta_accounts[~df_buy_insta_accounts.username.isin(user_text_prediction_df.user)][['username', 'biography']],
           df_women_event_mined[~df_women_event_mined.username.isin(user_text_prediction_df.user)][['username', 'biography']]
           ])
missing_users_df.rename(columns={'biography':'text'}, inplace=True)
missing_users_df = missing_users_df.reset_index()

In [53]:
missing_users_df = missing_users_df.dropna()

In [55]:
from collections import defaultdict
from transformers import AutoTokenizer
from datasets import Dataset
import pandas as pd

# using disilroberta as a quick baseline model
# https://huggingface.co/docs/transformers/tasks/sequence_classification
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)


def preprocess_for_inference_dataset(examples):
    examples = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    return examples


ds_inference = Dataset.from_pandas(missing_users_df[['text']])
tokenized_ds_inference = ds_inference.map(preprocess_for_inference_dataset)

Map: 100%|██████████| 6636/6636 [00:01<00:00, 5717.30 examples/s]


In [60]:
# loading the model previously trained
model_paths = [
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-agreeableness-training-text-350/checkpoint-1343",
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-conscientiousness-training-text-350/checkpoint-1580",
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-extraversion-training-text-350/checkpoint-1343",
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-neuroticism-training-text-350/checkpoint-711",
    "/Users/bohaocao/Codebase/uw-ocean/3.liwc_based_ft/models_350words/fine-tuned-regression-openness-training-text-350/checkpoint-1343"]
d = {}

for model_path in model_paths:
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    trait = model_path.split(sep="/")[-2].split(sep="-")[-4]
    # arguments for Trainer
    test_args = TrainingArguments(
        output_dir = model_path,
        do_train = False,
        do_predict = True,
        per_device_eval_batch_size = 50,   
        dataloader_drop_last = False    
    )

    # init trainer
    trainer = Trainer(
        model = model, 
        args = test_args, 
        compute_metrics = compute_metrics_for_regression
    )

    predictions = trainer.predict(tokenized_ds_inference)

    predictions_df = pd.DataFrame(predictions.predictions)
    d[trait]=predictions_df[0].tolist()

100%|██████████| 133/133 [02:08<00:00,  1.04it/s]


In [61]:
predictions_df = pd.DataFrame(d)

In [62]:
predictions_df

,agreeableness,conscientiousness,extraversion,neuroticism,openness
0,54.637936,48.007416,55.397968,34.497192,46.734192
1,52.568741,49.551327,54.891148,30.783730,46.234470
2,52.902840,47.403782,55.209686,35.656410,50.825588
3,53.144043,44.334667,55.492092,46.626801,47.678799
4,51.645348,43.946667,52.536003,27.142527,44.948498
...,...,...,...,...,...
6631,54.589378,42.740208,52.708797,31.985081,42.446053
6632,54.159027,38.006592,53.951298,27.847933,42.237133
6633,52.914593,36.122673,45.780075,28.005756,35.301163
6634,55.039314,49.554672,53.998188,36.343853,45.902111


In [63]:
missing_users_prediction_df = missing_users_df.merge(predictions_df,left_index=True, right_index=True)

In [66]:
missing_users_prediction_df

,index,username,text,agreeableness,conscientiousness,extraversion,neuroticism,openness,conscientiousness_prediction,openness_prediction,neuroticism_prediction,agreeableness_prediction,extraversion_prediction
0,4,houdaschimmel,Founder @shimmypetsalon @shimmyschimmel 🇲🇦|🇺🇸,54.637936,48.007416,55.397968,34.497192,46.734192,8,8,6,8,8
1,11,brookemasonphoto,"High Profile: Photo, Branding, Strategy Expert...",52.568741,49.551327,54.891148,30.783730,46.234470,8,8,5,7,8
2,20,jollymollydrip,👽CEO @jollymolly.custom 👾Modern Art Deep Minde...,52.902840,47.403782,55.209686,35.656410,50.825588,8,9,6,7,8
3,24,cassydy,"“Yes, that’s really how it’s spelled!”⁣⁣⁣⁣ Wit...",53.144043,44.334667,55.492092,46.626801,47.678799,7,8,9,7,8
4,26,katealuz,Los Angeles 🥰📍 Modelo internacional Vidente Te...,51.645348,43.946667,52.536003,27.142527,44.948498,7,8,4,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6630,10068,alli.mariant,💍Wife 🏃‍♂️🏃‍♂️🏃‍♀️Mommy of 3 💪🔥Team FitFire 💎D...,53.732468,45.528194,56.044453,28.728277,44.744968,8,8,5,7,8
6631,10069,stephanie_stack,Design-Creative Director.,54.589378,42.740208,52.708797,31.985081,42.446053,7,7,5,8,7
6632,10070,_goldendelicious,EMPOWERED WOMEN EMPOWER WOMEN Mother•Health Co...,54.159027,38.006592,53.951298,27.847933,42.237133,6,7,4,7,8
6633,10074,_unidegar,|SC🌴| NOLA⚜️| ATL 🍑 🇵🇦 @alignedstarsagency @do...,52.914593,36.122673,45.780075,28.005756,35.301163,6,6,4,7,6


In [65]:
# Convert the 0-100 numeric score into 1-10 categories.
traits = ['conscientiousness', 'openness', 'neuroticism','agreeableness', 'extraversion']
for trait in traits:
    missing_users_prediction_df[f'{trait}_prediction'] = pd.cut(missing_users_prediction_df[trait], 10, labels=[n for n in range(1, 11)])


In [107]:
df_women_event = pd.read_csv("liwc_results_350/df_women_event_prediction_350.csv")
df_buy_insta_accounts_add_on = pd.read_csv("liwc_results_350/df_buy_insta_accounts_add_on_prediction_350.csv")
df_buy_biz = pd.read_csv("liwc_results_350/df_buy_biz_prediction_350.csv")
df_buy_insta_accounts = pd.read_csv("liwc_results_350/df_buy_insta_accounts_prediction_350.csv")
df_women_event_mined = pd.read_csv("liwc_results_350/df_women_event_mined_prediction_350.csv")

In [82]:
missing_users_prediction_df.rename(columns={
    "conscientiousness_categorized":"conscientiousness_prediction",
    "openness_categorized":"openness_prediction",
    "neuroticism_categorized":"neuroticism_prediction",
    "agreeableness_categorized":"agreeableness_prediction",
    "extraversion_categorized":"extraversion_prediction"},inplace=True)

In [99]:
df_women_event_missing_user = df_women_event[df_women_event['conscientiousness_prediction'].isna()][columns_2].merge(missing_users_prediction_df[columns], left_on="username", right_on="username",how="left")

In [105]:
df_women_event = pd.concat([df_women_event[~df_women_event['conscientiousness_prediction'].isna()], df_women_event_missing_user]).drop_duplicates()

In [106]:
df_women_event.to_csv("liwc_results_350/df_women_event_prediction_350_with_no_post_users.csv")

In [116]:
columns = ['username','conscientiousness_prediction', 'openness_prediction', 'neuroticism_prediction','agreeableness_prediction', 'extraversion_prediction'] 
columns_2 = ['username', 'media_count', 'full_name', 'following_count',
       'follower_count', 'first_name', 'external_url', 'engagement_percent',
       'email', 'contact_phone_number', 'cityName', 'category', 'biography',
       'address_street', 'MostRecentPostDate', 'AvgLikes', 'AvgComments',
       'user']


def operate(df):
    columns_2 = list(set(df.columns) - set(columns))
    columns_2.append("username")
    df_missing_user = df[df['conscientiousness_prediction'].isna()][columns_2].merge(missing_users_prediction_df[columns], left_on="username", right_on="username",how="left")
    df = pd.concat([
        df[~df['conscientiousness_prediction'].isna()],
        df_missing_user
    ]).drop_duplicates()
    
    return df

In [110]:
df_women_event = operate(df_women_event)
df_women_event.to_csv("liwc_results_350/df_women_event_prediction_350_with_no_post_users.csv")

In [117]:
df_buy_insta_accounts_add_on = operate(df_buy_insta_accounts_add_on)
df_buy_insta_accounts_add_on.to_csv("liwc_results_350/df_buy_insta_accounts_add_on_prediction_350_with_no_post_users.csv")

df_buy_biz = operate(df_buy_biz)
df_buy_biz.to_csv("liwc_results_350/df_buy_biz_prediction_350_with_no_post_users.csv")

df_buy_insta_accounts = operate(df_buy_insta_accounts)
df_buy_insta_accounts.to_csv("liwc_results_350/df_buy_insta_accounts_prediction_350_with_no_post_users.csv")

df_buy_insta_accounts = operate(df_women_event_mined)
df_women_event_mined.to_csv("liwc_results_350/df_women_event_mined_prediction_350_with_no_post_users.csv")